# Tutorial 7: Patterns in the Darkness
## Pivot Tables and Crosstabs

---

### 📺 Video Walkthrough

Watch the video tutorial for this notebook: [YouTube Link](#)

---

### The Pattern

*"Three children in two months," the constable said. "All within a mile of the Quarry boundary. All disappeared between dusk and dawn."*

*Chief Archivist Mink spread the incident reports across her desk. "You think it's the creatures?"*

*"The trappers don't think so. They say the big predators don't come that close to the settlements. Too much light, too much noise."*

*"Then what?"*

*The constable looked toward the Quarry, visible through the archive's high windows. "The trappers talk about yellers. Things that move in groups. Two, three, five." He paused. "They say sometimes people become part of a yeller group."*

*Mink was quiet for a long moment. "Show me the data," she finally said. "All of it. Every incident, every disappearance, every creature sighting. If there's a pattern, we'll find it."*

*—From the investigation files, Capital Constabulary*

---

## What You Will Learn

In this tutorial, you will learn to:

1. Create pivot tables with `pd.pivot_table()`
2. Use crosstabs for frequency analysis
3. Apply multiple aggregation functions
4. Add margins (totals) to your tables
5. Find patterns in categorical data

By the end, you will be able to answer questions like:
- What types of incidents occur most often in each sector?
- Is there a relationship between time of day and incident severity?
- What patterns exist in the disappearance data?

---

In [ ]:
import pandas as pd
import numpy as np

# Load data from GitHub
BASE_URL = "https://raw.githubusercontent.com/YOUR_USERNAME/yeller-quarry-data-science/main/data/"

incidents = pd.read_csv(BASE_URL + "incidents.csv")
catches = pd.read_csv(BASE_URL + "catches.csv")
creatures = pd.read_csv(BASE_URL + "creatures.csv")
crews = pd.read_csv(BASE_URL + "crews.csv")

# Convert dates
incidents['date'] = pd.to_datetime(incidents['date'])
catches['date'] = pd.to_datetime(catches['date'])

# Add calculated columns
catches['total_value'] = catches['quantity'] * catches['price_per_unit']
incidents['month'] = incidents['date'].dt.month_name()

print("Data loaded.")
print(f"  Incidents: {len(incidents)} records")
print(f"  Catches: {len(catches)} records")

In [ ]:
# Examine the incident data
incidents.head(10)

In [ ]:
# What types of incidents do we have?
incidents['incident_type'].value_counts()

Animal attacks are the most common, but there are also disappearances, injuries, equipment failures, and something called "yeller_phenomenon."

---

## Part 1: Basic Pivot Tables

A pivot table summarizes data by grouping it along two dimensions—rows and columns.

In [ ]:
# Count incidents by type and sector
pivot1 = pd.pivot_table(
    incidents,
    index='incident_type',      # Rows
    columns='sector',           # Columns
    values='incident_id',       # What to count
    aggfunc='count',            # How to aggregate
    fill_value=0                # Replace NaN with 0
)

pivot1

This immediately reveals patterns:
- **Animal attacks** cluster in the Cave Interior and Deep Tunnels
- **Disappearances** happen at the Quarry Boundary—not deep inside
- **Yeller phenomena** occur in Western Marsh

The disappearances at the boundary are concerning. These aren't trappers going too deep—these are people near the edge.

---

## Part 2: Pivot Table Parameters

The `pivot_table()` function has several useful parameters:

```python
pd.pivot_table(
    data,              # DataFrame
    index='row_var',   # Variable for rows
    columns='col_var', # Variable for columns
    values='value',    # What to aggregate
    aggfunc='mean',    # Aggregation function
    fill_value=0,      # Replace missing values
    margins=True       # Add row/column totals
)
```

In [ ]:
# Same pivot with margins (totals)
pivot2 = pd.pivot_table(
    incidents,
    index='incident_type',
    columns='sector',
    values='incident_id',
    aggfunc='count',
    fill_value=0,
    margins=True,           # Add totals
    margins_name='TOTAL'    # Label for totals
)

pivot2

---

## Part 3: Crosstabs

Crosstabs are a simpler way to count frequencies between two categorical variables.

In [ ]:
# Crosstab of incident type vs severity
crosstab1 = pd.crosstab(
    incidents['incident_type'],
    incidents['severity'],
    margins=True
)

crosstab1

The fatal incidents: 2 from animal attacks, 1 from disappearance (presumed dead). 

Truck's death is in there. And The Boss's disappearance, classified as "critical" because her body was never found.

### Crosstab with percentages

In [ ]:
# Row percentages (what % of each incident type has each severity)
crosstab_pct = pd.crosstab(
    incidents['incident_type'],
    incidents['severity'],
    normalize='index'  # 'index' = row %, 'columns' = col %, 'all' = total %
) * 100

crosstab_pct.round(1)

25% of animal attacks are fatal. Disappearances are 20% fatal (presumed dead) and 40% critical (missing, unknown status). 

Equipment failures are never fatal—they're annoying, but they don't kill.

---

## Part 4: Multiple Aggregations

Pivot tables can apply multiple functions to the same values.

In [ ]:
# Catch analysis by sector
catch_pivot = pd.pivot_table(
    catches,
    index='sector',
    values=['quantity', 'total_value'],
    aggfunc={'quantity': ['sum', 'mean'], 'total_value': ['sum', 'mean']}
).round(1)

catch_pivot

The Deep Tunnels and Grimslew Shore have the highest average catch values—dangerous areas with valuable specimens. The Western Marsh has high total quantity but lower average value—abundant but less exotic creatures.

---

## Part 5: Multi-Index Pivots

You can have multiple levels of index.

In [ ]:
# Incidents by sector AND severity, with type as columns
multi_pivot = pd.pivot_table(
    incidents,
    index=['sector', 'severity'],
    columns='incident_type',
    values='incident_id',
    aggfunc='count',
    fill_value=0
)

multi_pivot

---

## Part 6: The Disappearance Pattern

Let's focus on the disappearances.

In [ ]:
# Filter to disappearances only
disappearances = incidents[incidents['incident_type'] == 'disappearance']
print(f"Total disappearances: {len(disappearances)}")
disappearances[['date', 'sector', 'severity', 'description', 'crew_involved']]

In [ ]:
# When do disappearances happen?
disappearances['month'] = disappearances['date'].dt.month_name()
pd.crosstab(disappearances['sector'], disappearances['month'])

The Quarry Boundary disappearances cluster in February and March. The Boss went missing in March, but she was in the Cave Interior—a different pattern.

Let's look at the descriptions more closely:

In [ ]:
# Boundary disappearances
boundary_cases = disappearances[disappearances['sector'] == 'Quarry Boundary']
for _, row in boundary_cases.iterrows():
    print(f"{row['date'].strftime('%Y-%m-%d')}: {row['description']}")
    print(f"  Severity: {row['severity']}")
    print()

Three children. All near the boundary. All between dusk and dawn.

The trappers talk about yellers—things that move in groups of 2, 3, 5, 7. The archivists say anything living can become a yeller group.

*Sometimes people become part of a yeller group.*

---

## Part 7: Catch Patterns

Let's use pivot tables on the catch data.

In [ ]:
# Merge catches with creature info
catches_full = catches.merge(
    creatures[['creature_id', 'common_name', 'category']],
    on='creature_id'
)

# Pivot: category vs sector
catch_by_category = pd.pivot_table(
    catches_full,
    index='category',
    columns='sector',
    values='total_value',
    aggfunc='sum',
    fill_value=0,
    margins=True
)

catch_by_category

In [ ]:
# Live vs Dead by sector
condition_pivot = pd.pivot_table(
    catches_full,
    index='sector',
    columns='condition',
    values='total_value',
    aggfunc='sum',
    fill_value=0
)

condition_pivot['live_ratio'] = condition_pivot['live'] / (condition_pivot['live'] + condition_pivot['dead'])
condition_pivot.round(2)

The Western Marsh has the highest proportion of live catches—the creatures there can be captured without killing them. The Deep Tunnels are almost all dead catches—you don't bring a Maw Beast back alive.

---

## Part 8: Time-Based Pivots

In [ ]:
# Add month to catches
catches_full['month'] = catches_full['date'].dt.month_name()

# Revenue by month and category
monthly_category = pd.pivot_table(
    catches_full,
    index='month',
    columns='category',
    values='total_value',
    aggfunc='sum',
    fill_value=0
)

# Reorder months chronologically
month_order = ['January', 'February', 'March', 'April']
monthly_category = monthly_category.reindex(month_order)
monthly_category

March had the highest bird revenue—that's the yeller bird flock for Senator Huilof. April sees increased aquatic catches as the spring thaw opens up the water channels.

---

## Part 9: Heatmap Visualization

In [ ]:
import matplotlib.pyplot as plt

# Create a heatmap of incidents
incident_matrix = pd.pivot_table(
    incidents,
    index='incident_type',
    columns='sector',
    values='incident_id',
    aggfunc='count',
    fill_value=0
)

plt.figure(figsize=(12, 6))
plt.imshow(incident_matrix.values, cmap='YlOrRd', aspect='auto')
plt.colorbar(label='Number of Incidents')
plt.xticks(range(len(incident_matrix.columns)), incident_matrix.columns, rotation=45, ha='right')
plt.yticks(range(len(incident_matrix.index)), incident_matrix.index)
plt.title('Incident Frequency by Type and Sector')

# Add numbers in cells
for i in range(len(incident_matrix.index)):
    for j in range(len(incident_matrix.columns)):
        plt.text(j, i, str(incident_matrix.values[i, j]), 
                ha='center', va='center', fontsize=12)

plt.tight_layout()
plt.show()

---

## ✏️ Exercises

### Exercise 1: Crew Incident Rates

Create a pivot table showing how many incidents of each type occurred for each crew. Which crew has the most incidents? The most serious incidents?

In [ ]:
# Your code here



### Exercise 2: Destination Analysis

Using the catches data, create a crosstab showing which destinations receive which creature categories. Where do birds go? Reptiles?

In [ ]:
# Your code here



### Exercise 3: Yeller Quantities

The yeller numbers are 2, 3, 5, 7, and 11. Create a new column in catches indicating whether the quantity is a "yeller number." Then create a crosstab showing yeller vs non-yeller catches by creature category.

In [ ]:
# Your code here



### Exercise 4: Monthly Incident Severity

Create a pivot table showing average incident severity by month and sector. (You'll need to convert severity to numbers first: minor=1, moderate=2, serious=3, critical=4, fatal=5)

In [ ]:
# Your code here



### Exercise 5: The Investigation

Chief Archivist Mink wants a complete report on the Quarry Boundary disappearances. Create a summary that includes:
- Total number of disappearances
- Dates of all boundary disappearances
- Common characteristics (time of day if mentioned, ages of victims, etc.)
- Any connection to yeller phenomena

Present your findings as if you were reporting to the Constabulary.

In [ ]:
# Your code here



---

## 📋 Summary

In this tutorial, you learned:

| Concept | Code |
|---------|------|
| Basic pivot table | `pd.pivot_table(df, index='a', columns='b', values='c')` |
| With aggregation | `pd.pivot_table(..., aggfunc='sum')` |
| With margins | `pd.pivot_table(..., margins=True)` |
| Multiple aggregations | `pd.pivot_table(..., aggfunc=['sum', 'mean'])` |
| Crosstab | `pd.crosstab(df['a'], df['b'])` |
| Crosstab with percentages | `pd.crosstab(..., normalize='index')` |
| Fill missing values | `pd.pivot_table(..., fill_value=0)` |

---

## ➡️ Next Tutorial

In **Tutorial 8: Cleaning the Archives**, you will learn to handle messy data—missing values, duplicates, inconsistent formatting, and the kind of errors that creep into real-world datasets.

*"The problem with Quarry records," Mink said, "is that they're written by trappers. Half of them can't spell. The other half lie."*

*"Then how do you know what's true?"*

*"You look for patterns. Liars make mistakes. The truth is consistent—even when it's strange."*

---

## 💾 Save Your Work

**File > Save a copy in Drive** to keep your completed notebook.